In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import scipy

In [2]:
import seaborn as sns

In [3]:
import sys
sys.path.append('..')

from Approximators.Bernstein import CauchySimplex

In [4]:
from utils import solve_single_coefficient_bessel

Consider Bessel's differential equation with initial conditions $y(1)=y(e^a)=0$. Let $J_m(x)$ and $Y_m(x)$ be Bessel functions of the first and second kind, respectively. Then, this differential equation has solutions of the form
\begin{align}
    y\ =\ c_1\, J_m(\sqrt{\lambda}\, x)\ +\ c_2\, Y_m(\sqrt{\lambda}\, x),
\end{align}
for constants $c_1, c_2$. To satisfy the initial conditions, substitution and solving for $c_1$ would show that if $\lambda$ is an eigenvalue, it must satisfy
\begin{align}
    \frac{J_m(\sqrt{\lambda}\, e^a)\,Y_m(\sqrt{\lambda})}{J_m(\sqrt{\lambda})\,Y_m(\sqrt{\lambda}\, e^a)}\ =\ 1.
\end{align}

Taking the parameterization $x=e^{az}$ yields the differential equation
\begin{align}
    (\lambda\, e^{2ax}\ -\ m^2) y\ +\ \frac{1}{a^2}y''\ =\ 0,
\end{align}
with initial conditions $y(0)=y(1)=0$. Thus yielding an eigenvalue problem with one nonconstant coefficient.

In [5]:
m = 2
a = 4

In [6]:
x = np.linspace(0, 1, 256)
y = np.exp(2 * a * x)

In [7]:
n_eigenvals = 20

In [8]:
n_vals = np.arange(4, 20 + 1, 1)

In [9]:
Nx = 2 ** 12
Nx

4096

# Polynomial Approximation

In [10]:
polynomial_results = []

for n in n_vals:
    print(f"Starting n = {n}")
    
    polynomial_approximator = CauchySimplex(n, 0).fit(x, y)
    
    time_taken, solver = solve_single_coefficient_bessel(polynomial_approximator, a, m, Lx=0, Ux=1, Nx=Nx, 
                                                         dtype=np.float64, n_eigenvals=n_eigenvals, n_runs=5)
    
    evals = np.sort(solver.eigenvalues.real)
    
    ratio = scipy.special.jv(m, np.sqrt(evals)) * scipy.special.yv(m, np.sqrt(evals) * np.exp(a)) \
            / (scipy.special.jv(m, np.sqrt(evals) * np.exp(a)) * scipy.special.yv(m, np.sqrt(evals)))

    y_pred = polynomial_approximator(x)
    approximation_error = np.linalg.norm(y_pred - y)
    
    ratio_dataframe = pd.DataFrame(abs(ratio - 1), columns=['Eigenvalue Ratio Error'])
    ratio_dataframe['Approximator'] = 'Polynomial'
    ratio_dataframe['Approximation Error'] = approximation_error
    ratio_dataframe['Num. Coefs'] = n
    ratio_dataframe['Time'] = time_taken
    
    polynomial_results.append(ratio_dataframe)

Starting n = 4
2023-10-05 12:39:49,446 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 2.1e+00/s


/var/folders/99/5f_qszrn5y132sfcdd12k2f80000gn/T/ipykernel_8155/2280594852.py:13: RuntimeWarning: invalid value encountered in sqrt
  ratio = scipy.special.jv(m, np.sqrt(evals)) * scipy.special.yv(m, np.sqrt(evals) * np.exp(a)) \
/var/folders/99/5f_qszrn5y132sfcdd12k2f80000gn/T/ipykernel_8155/2280594852.py:14: RuntimeWarning: invalid value encountered in sqrt
  / (scipy.special.jv(m, np.sqrt(evals) * np.exp(a)) * scipy.special.yv(m, np.sqrt(evals)))


Starting n = 5
2023-10-05 12:39:50,499 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.9e+00/s
Starting n = 6
2023-10-05 12:39:51,519 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.9e+00/s
Starting n = 7
2023-10-05 12:39:52,610 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.7e+00/s
Starting n = 8
2023-10-05 12:39:53,710 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.7e+00/s
Starting n = 9
2023-10-05 12:39:54,828 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.6e+00/s
Starting n = 10
2023-10-05 12:39:56,032 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.4e+00/s
Starting n = 11
2023-10-05 12:39:57,252 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, 

# Rational Approximation

In [11]:
rational_results = []

for n in n_vals:
    print(f"Starting n = {n}")

    if n < 15:
        rational_approximator = CauchySimplex(n, n, hot_start=True, max_iter=500, 
                                              gamma=0.9).fit(x, y)
    else:
        rational_approximator = CauchySimplex(n, n, hot_start=False, max_iter=500, 
                                              gamma=0.9, stopping_tol=0).fit(x, y)
    
    time_taken, solver = solve_single_coefficient_bessel(rational_approximator, a, m, Lx=0, Ux=1, Nx=Nx, 
                                                         dtype=np.float64, n_eigenvals=20, n_runs=5)
    
    evals = np.sort(solver.eigenvalues.real)
    
    ratio = scipy.special.jv(m, np.sqrt(evals)) * scipy.special.yv(m, np.sqrt(evals) * np.exp(a)) \
            / (scipy.special.jv(m, np.sqrt(evals) * np.exp(a)) * scipy.special.yv(m, np.sqrt(evals)))
    
    y_pred = rational_approximator(x)
    approximation_error = np.linalg.norm(y_pred - y)

    ratio_dataframe = pd.DataFrame(abs(ratio - 1), columns=['Eigenvalue Ratio Error'])
    ratio_dataframe['Approximator'] = 'Rational'
    ratio_dataframe['Approximation Error'] = approximation_error
    ratio_dataframe['Num. Coefs'] = n
    ratio_dataframe['Time'] = time_taken
    
    rational_results.append(ratio_dataframe)

Starting n = 4
2023-10-05 12:40:11,651 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.8e+00/s
Starting n = 5
2023-10-05 12:40:12,849 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.5e+00/s
Starting n = 6
2023-10-05 12:40:14,100 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.4e+00/s
Starting n = 7
2023-10-05 12:40:17,693 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.3e+00/s
Starting n = 8
2023-10-05 12:40:23,296 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.2e+00/s
Starting n = 9
2023-10-05 12:40:24,787 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.1e+00/s
Starting n = 10
2023-10-05 12:40:26,312 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, R

# Plots

In [12]:
results_df = pd.concat(polynomial_results + rational_results)

In [13]:
results_df.head()

,Eigenvalue Ratio Error,Approximator,Approximation Error,Num. Coefs,Time
0,NaN,Polynomial,555.272752,4,0.242821
1,1.000139,Polynomial,555.272752,4,0.242821
2,0.999466,Polynomial,555.272752,4,0.242821
3,0.999197,Polynomial,555.272752,4,0.242821
4,0.997272,Polynomial,555.272752,4,0.242821


In [14]:
average_ratio = abs(results_df.groupby(['Num. Coefs', 'Approximator']).mean())

average_ratio = average_ratio['Eigenvalue Ratio Error'].copy()
average_ratio.name = 'Eigenvalue Ratio Error'

In [15]:
approximation_error = results_df.groupby(['Num. Coefs', 'Approximator']).mean()['Approximation Error']

time_taken = results_df.groupby(['Num. Coefs', 'Approximator']).mean()['Time']
time_taken.name = 'Time (sec)'

In [16]:
results = pd.concat([average_ratio, approximation_error, time_taken], axis=1)
results = results.pivot_table(index='Num. Coefs', columns='Approximator')

In [17]:
column_order = [(col_name, approximator_type) 
                for col_name in ['Eigenvalue Ratio Error', 'Approximation Error', 'Time (sec)']
                for approximator_type in ['Polynomial', 'Rational']]

In [18]:
results = results.loc[:, column_order]

In [19]:
results

Eigenvalue Ratio Error               Approximation Error  \
Approximator             Polynomial      Rational          Polynomial   
Num. Coefs                                                              
4                      1.009375e+00  8.547040e-03        5.552728e+02   
5                      9.709918e-01  3.059161e-05        1.708277e+02   
6                      1.084594e+00  2.866464e-07        4.593033e+01   
7                      3.334697e+00  8.994893e-06        1.095105e+01   
8                      5.468155e-01  3.067684e-06        2.343200e+00   
9                      1.709507e-01  1.779364e-07        4.544228e-01   
10                     1.468125e-02  2.902204e-08        8.054182e-02   
11                     3.351538e-03  7.725405e-09        1.313905e-02   
12                     4.607940e-04  2.149476e-08        1.984840e-03   
13                     4.148658e-05  1.170742e-08        2.791210e-04   
14                     3.719098e-06  1.479714e-07        3.670831e-05   
15                     6.186236e-07  1.557391e-08        4.533177e-06   
16                     6.534123e-08  4.688327e-09        5.275581e-07   
17                     4.472377e-09  1.905913e-08        5.804474e-08   
18                     4.451225e-10  2.010802e-09        6.055588e-09   
19                     1.729838e-10  1.470985e-10        6.030244e-10   
20                     2.300126e-10  5.119984e-10        9.902012e-11   

                           Time (sec)            
Approximator      Rational Polynomial  Rational  
Num. Coefs                                       
4             2.370116e-02   0.242821  0.213540  
5             2.360691e-04   0.203801  0.233623  
6             1.643591e-06   0.201102  0.243548  
7             5.893155e-05   0.215873  0.260610  
8             2.070101e-05   0.215777  0.266366  
9             9.435233e-07   0.221580  0.292764  
10            2.688540e-07   0.239414  0.297322  
11            6.666913e-08   0.238419  0.322058  
12            1.191888e-07   0.251918  0.330979  
13            1.031848e-07   0.252107  0.358199  
14            1.558012e-06   0.267131  0.369251  
15            1.974167e-07   0.268435  0.397978  
16            5.161350e-08   0.295124  0.400566  
17            1.392351e-08   0.319384  0.387579  
18            6.054913e-09   0.314581  0.306800  
19            6.024016e-10   0.339985  0.306929  
20            8.103168e-11   0.319141  0.325771

In [20]:
formatters = [lambda x: f"{x:.4e}"] * 4\
                + [lambda x: f"{x:.4f}"] * 2
print(results.to_latex(formatters=formatters))

\begin{tabular}{lrrrrrr}
\toprule
 & \multicolumn{2}{r}{Eigenvalue Ratio Error} & \multicolumn{2}{r}{Approximation Error} & \multicolumn{2}{r}{Time (sec)} \\
Approximator & Polynomial & Rational & Polynomial & Rational & Polynomial & Rational \\
Num. Coefs &  &  &  &  &  &  \\
\midrule
4 & 1.0094e+00 & 8.5470e-03 & 5.5527e+02 & 2.3701e-02 & 0.2428 & 0.2135 \\
5 & 9.7099e-01 & 3.0592e-05 & 1.7083e+02 & 2.3607e-04 & 0.2038 & 0.2336 \\
6 & 1.0846e+00 & 2.8665e-07 & 4.5930e+01 & 1.6436e-06 & 0.2011 & 0.2435 \\
7 & 3.3347e+00 & 8.9949e-06 & 1.0951e+01 & 5.8932e-05 & 0.2159 & 0.2606 \\
8 & 5.4682e-01 & 3.0677e-06 & 2.3432e+00 & 2.0701e-05 & 0.2158 & 0.2664 \\
9 & 1.7095e-01 & 1.7794e-07 & 4.5442e-01 & 9.4352e-07 & 0.2216 & 0.2928 \\
10 & 1.4681e-02 & 2.9022e-08 & 8.0542e-02 & 2.6885e-07 & 0.2394 & 0.2973 \\
11 & 3.3515e-03 & 7.7254e-09 & 1.3139e-02 & 6.6669e-08 & 0.2384 & 0.3221 \\
12 & 4.6079e-04 & 2.1495e-08 & 1.9848e-03 & 1.1919e-07 & 0.2519 & 0.3310 \\
13 & 4.1487e-05 & 1.1707e-08 & 2.79